In [7]:
import requests
from bs4 import BeautifulSoup
from rake_nltk import Rake
rake_nltk_var = Rake()

scraping_url = "https://www.cs.hku.hk/programmes/course-offered"

re = requests.get(scraping_url)
soup = BeautifulSoup(re.text, "html.parser")

In [8]:
course_table = soup.find_all('tr')
found_flag = 0
for idx, tr in enumerate(course_table):
  if tr.text[1:-1] == "Year 2 to 4 - Elective":
    found_flag = idx
    break

elective_table = course_table[found_flag+1:]

In [ ]:
import json

elective_data = {}


for elective in elective_table:
  td = elective.find_all('td')
  code = td[0].text
  link = td[1].find('a')
  course_name = link.text
  course_link = "www.cs.hku.hk" + link['href']
  course_data = {}
  course_data['course_code'] =  str(code)
  course_data['course_name'] = course_name
  course_data['course_link'] = 'https://' + course_link
  elective_data[code] = course_data

y = json.dumps(elective_data)
print(elective_data)


In [12]:
course_keywords = {}

for course in elective_data:
  course_object = elective_data.get(course)

  re = requests.get(course_object.get('course_link'))
  soup = BeautifulSoup(re.text, "html.parser")

  page_tables = soup.find_all('table', {
      'class': 'table_listdata_v4 table-responsive'
  })

  course_syllabus = page_tables[-1].find_all()
  course_text = []
  for i in range(0, len(course_syllabus)):
    if('\n' not in course_syllabus[i].text and not course_syllabus[i].text.isdigit()):
      course_text.append(course_syllabus[i].text)
  course_text = ','.join(course_text)
  rake_nltk_var.extract_keywords_from_text(course_text)
  keyword_extracted = rake_nltk_var.get_ranked_phrases()
  course_keywords[course] = keyword_extracted
  course_keywords[course] = set(course_keywords[course]) #for data cleaning
  


In [16]:
for course in course_keywords:
  course_keywords[course] = list(course_keywords[course])
print(course_keywords)

{'COMP2501': ['data acquisition', 'data source', 'data visualization tools', 'data science', 'statistical inference', 'data manipulation', 'processing', 'feature engineering', 'exploratory data analysis', 'data pre', 'data preparation', 'manipulation', 'regression models', 'implementation', 'python', 'machine learning', 'visualization', 'data analysis', 'communication'], 'COMP2502': ['data structures library', 'oriented programming language', 'data structures', 'computing fundamentals', 'manipulate', 'object', 'applying data structures', 'basic algorithms', 'oriented programming principles', 'basic data structures'], 'COMP3231': ['cycle processors', 'design', 'cache', 'virtual memory', 'computer architecture evaluation', 'instruction set architecture', 'logic design', 'output', 'computer design', 'ic', 'basics', 'performance analysis', 'trends', 'multi', 'computer architecture design', 'cost', 'quantitative principles', 'power', 'main memory', 'computer architecture fundamentals', 'tec

In [ ]:
import nltk
nltk.download('punkt')

In [18]:
y= json.dumps(course_keywords)
print(y)

{"COMP2501": ["data acquisition", "data source", "data visualization tools", "data science", "statistical inference", "data manipulation", "processing", "feature engineering", "exploratory data analysis", "data pre", "data preparation", "manipulation", "regression models", "implementation", "python", "machine learning", "visualization", "data analysis", "communication"], "COMP2502": ["data structures library", "oriented programming language", "data structures", "computing fundamentals", "manipulate", "object", "applying data structures", "basic algorithms", "oriented programming principles", "basic data structures"], "COMP3231": ["cycle processors", "design", "cache", "virtual memory", "computer architecture evaluation", "instruction set architecture", "logic design", "output", "computer design", "ic", "basics", "performance analysis", "trends", "multi", "computer architecture design", "cost", "quantitative principles", "power", "main memory", "computer architecture fundamentals", "tec